<a href="https://colab.research.google.com/github/SitianXiong/Tempoary_CNN/blob/master/Correct_labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Authenticate and initialize GEE
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=eeKDAFDlxOtqGSrvANsnu2P-PW4qfoeOUDWhA14WcVc&tc=CP3rjs_VjML0xVPcJzOCiBYodO8a-RET2NIK01bm6Z4&cc=9Ou6ZpiNkQJOW64yFpFt9ATcvkyuZOCdts-mkI8DteQ

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qjalf2UTORuzlO0NtT8lltHTDdSW2jRTXc0wMrPti0bpjel0v852rI

Successfully saved authorization token.


## Here we define a python GEE API version of list_dir()

In [23]:
# define a python version of Sitian's list_dir()

def list_dir(dir):
  # list files in old dir
    obj_list = []
    f = ee.data.listAssets({'parent':dir,'pageSize':1500})['assets']
    for i in range(len(f)):
        name = f[i]['name'] #get obj name (not full path)
        obj_list.append(name)
  
    return obj_list
  

## Here we import the zamtile_i (tile bbox), original field boudaries, and revised field boundaries

In [34]:
import os

#No need to import ee here because you imported it in previous cell.
#import ee

zamtile_i = ee.FeatureCollection("users/stxiong8/Zamtile_600_i")

# I tried to create a function called 'correct_labels'. This part should put in between 6 and 7.

# path = 'users/stxiong8/Utilities:manage_asset'
# def correct_labels(path):
#   dir_list = os.listdir(path) 
#   file_list=[export_ft]
#   i=0
#   for i in len(os.listdir(path)):
#       if ee.FeatureCollection(merge_fields4).filterBounds(tile.geometry()) = ee.Feature(ft.geometry()):
#         os.rename(fields_in_a_tile, tile_name)
#       i+=1
#       print(fields_in_a_tile)


# From here, I only took 'var' away from original scripts in GEE

# WE USE THE PYTHON VERSION OF list_dir defined above
#list_dir = ('users/stxiong8/Utilities:manage_asset').list_dir
all_fields = list_dir('projects/zamlc2020unet/assets/AEZ_I/')
revise_fields = list_dir('projects/zamlc2020unet/assets/AEZ_I_revise/')

# you can print the all_fields.  It should be a
# list of ids of field boundary tiles
#print(all_fields)
#print(revise_fields)

zamtile_i = ee.FeatureCollection(zamtile_i.toList(5))
print("We have imported ({}) of zamtile_i, ({}) of original field boundary tiles, and ({}) revised field boudary tiles.".format(len(zamtile_i.getInfo()), len(all_fields), len(revise_fields)))


57
We have imported (3) of zamtile_i, (514) of original field boundary tiles, and (57) revised field boudary tiles.


## Now we do the rest of things except for the data export

In [164]:


def poly_to_ft(coords):
  return ee.Feature(ee.Geometry.Polygon(coords))

#2. merge all field boundaries into featurecollections
all_fields1 = ee.FeatureCollection(list(map(lambda x: ee.FeatureCollection(x), all_fields)))
revise_fields1 = ee.FeatureCollection(list(map(lambda x: ee.FeatureCollection(x), revise_fields)))

# make a single feature collection
all_fields2 = ee.FeatureCollection(all_fields1).flatten()

revise_fields2 = ee.FeatureCollection(revise_fields1).flatten()
#print(revise_fields1.getInfo())

#3. merge 2 fields featurecollections
merge_fields = all_fields2.merge(revise_fields2)

# 4. merge fields and tiles featurecollection

merge_fields_tiles = merge_fields.merge(zamtile_i)


# 5. convert merge_fields_tiles featurecollection to feature 
merge_fields3 = merge_fields.geometry()

#Here we convert each field (a closed circle) to a feature

merge_fields4 = merge_fields3.coordinates().map(poly_to_ft)#ee.Feature(merge_fields3)
print(merge_fields4.size().getInfo(),'merge_fields4')



# 6. correct fields names
def correct_name(tile): 
    ft = ee.FeatureCollection(merge_fields4).filterBounds(tile.geometry())
    ft = ee.Feature(ft.geometry())
    ft = ft.set({'name': tile.get('name')})  
    return ft
  


 
# we loop on all tiles and export_ft will be a featurecollection, which each feature is a field boudaries that within the corresponding tile 
export_ft = zamtile_i.map(correct_name,True)
print(export_ft.size().getInfo())





3771 merge_fields4
5


## Export 


In [166]:
l = export_ft.size().getInfo()
print(l)
for i in range(l):

    fields_in_a_tile =  ee.Feature(export_ft.toList(1e6).get(i))
    tile_name = fields_in_a_tile.get('name').getInfo()
  

    fields_in_a_tile = ee.FeatureCollection([fields_in_a_tile])

    task = ee.batch.Export.table.toAsset(collection=fields_in_a_tile, 
                                          description=tile_name, 
                                          assetId= 'projects/zamlc2020unet/assets/AEZ_I_test_Sitian/'+tile_name
                                        )


    task.start()
  


5
